# EDA

### 生成Data_csv并保存

In [3]:
# 导入库
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
root_path = './'

In [3]:
data_csv_raw = pd.read_csv(os.path.join(root_path, 'train.csv'))
print(data_csv_raw.head())

                         id        class segmentation
0  case123_day20_slice_0001  large_bowel          NaN
1  case123_day20_slice_0001  small_bowel          NaN
2  case123_day20_slice_0001      stomach          NaN
3  case123_day20_slice_0002  large_bowel          NaN
4  case123_day20_slice_0002  small_bowel          NaN


In [4]:
def get_path(str_id):
    root_path = './'
    train_root_path = os.path.join(root_path, 'train')
    str_id_list = str_id.split('_')
    case_path = os.path.join(train_root_path, str_id_list[0])
    day_path = os.path.join(case_path, str_id_list[0] + '_' + str_id_list[1])
    slice_path_list = os.listdir(os.path.join(day_path, 'scans'))
    slice_path_10_list = []
    for item in slice_path_list:
        slice_path_10_list.append(item[:10])
    index = slice_path_10_list.index(str_id_list[2] + '_' + str_id_list[3])
    path = os.path.join(day_path, slice_path_list[index])
    return path
    
def get_day_id(str_id):
   str_id_list = str_id.split('_')
   return str_id_list[1][3:]

In [5]:
get_path('case123_day20_slice_0001')

'./train\\case123\\case123_day20\\slice_0001_266_266_1.50_1.50.png'

In [6]:
data_csv = pd.DataFrame(columns=['id', 'path', 'segmentation_s',
                                     'segmentation_lb','segmentation_sb',  
                                     'case_id', 'day_id', 'slice_h', 
                                     'slice_w', 'px_spacing_h', 
                                     'px_spacing_w', 'slice_id'])
data_csv['id'] = data_csv_raw.id.unique()

                         id path segmentation_s segmentation_lb  \
0  case123_day20_slice_0001  NaN            NaN             NaN   
1  case123_day20_slice_0002  NaN            NaN             NaN   
2  case123_day20_slice_0003  NaN            NaN             NaN   
3  case123_day20_slice_0004  NaN            NaN             NaN   
4  case123_day20_slice_0005  NaN            NaN             NaN   

  segmentation_sb case_id day_id slice_h slice_w px_spacing_h px_spacing_w  \
0             NaN     NaN    NaN     NaN     NaN          NaN          NaN   
1             NaN     NaN    NaN     NaN     NaN          NaN          NaN   
2             NaN     NaN    NaN     NaN     NaN          NaN          NaN   
3             NaN     NaN    NaN     NaN     NaN          NaN          NaN   
4             NaN     NaN    NaN     NaN     NaN          NaN          NaN   

  slice_id  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN  


In [7]:
data_csv['path'] = data_csv['id'].apply(get_path)
data_csv['case_id'] = data_csv['id'].apply(lambda x: x[4:x.index('_')])
data_csv['day_id'] = data_csv['id'].apply(get_day_id)
data_csv['slice_h'] = data_csv['path'].apply(lambda x: x[-21:-18])
data_csv['slice_w'] = data_csv['path'].apply(lambda x: x[-17:-14])
data_csv['px_spacing_h'] = data_csv['path'].apply(lambda x: x[-13:-9])
data_csv['px_spacing_w'] = data_csv['path'].apply(lambda x: x[-8:-4])
data_csv['slice_id'] = data_csv['id'].apply(lambda x: x[-4:])
data_csv
# data_csv.fillna('', inplace=True)

,id,path,segmentation_s,segmentation_lb,segmentation_sb,case_id,day_id,slice_h,slice_w,px_spacing_h,px_spacing_w,slice_id
0,case123_day20_slice_0001,./train\case123\case123_day20\slice_0001_266_2...,NaN,NaN,NaN,123,20,266,266,1.50,1.50,0001
1,case123_day20_slice_0002,./train\case123\case123_day20\slice_0002_266_2...,NaN,NaN,NaN,123,20,266,266,1.50,1.50,0002
2,case123_day20_slice_0003,./train\case123\case123_day20\slice_0003_266_2...,NaN,NaN,NaN,123,20,266,266,1.50,1.50,0003
3,case123_day20_slice_0004,./train\case123\case123_day20\slice_0004_266_2...,NaN,NaN,NaN,123,20,266,266,1.50,1.50,0004
4,case123_day20_slice_0005,./train\case123\case123_day20\slice_0005_266_2...,NaN,NaN,NaN,123,20,266,266,1.50,1.50,0005
...,...,...,...,...,...,...,...,...,...,...,...,...
38491,case30_day0_slice_0140,./train\case30\case30_day0\slice_0140_266_266_...,NaN,NaN,NaN,30,0,266,266,1.50,1.50,0140
38492,case30_day0_slice_0141,./train\case30\case30_day0\slice_0141_266_266_...,NaN,NaN,NaN,30,0,266,266,1.50,1.50,0141
38493,case30_day0_slice_0142,./train\case30\case30_day0\slice_0142_266_266_...,NaN,NaN,NaN,30,0,266,266,1.50,1.50,0142
38494,case30_day0_slice_0143,./train\case30\case30_day0\slice_0143_266_266_...,NaN,NaN,NaN,30,0,266,266,1.50,1.50,0143


In [9]:
data_csv_raw_ics = data_csv_raw[['id', 'class', 'segmentation']]
data_csv_raw_ics

,id,class,segmentation
0,case123_day20_slice_0001,large_bowel,NaN
1,case123_day20_slice_0001,small_bowel,NaN
2,case123_day20_slice_0001,stomach,NaN
3,case123_day20_slice_0002,large_bowel,NaN
4,case123_day20_slice_0002,small_bowel,NaN
...,...,...,...
115483,case30_day0_slice_0143,small_bowel,NaN
115484,case30_day0_slice_0143,stomach,NaN
115485,case30_day0_slice_0144,large_bowel,NaN
115486,case30_day0_slice_0144,small_bowel,NaN


In [10]:
for item in range(int(len(data_csv_raw_ics)/3)):
    data_csv.loc[item, 'segmentation_lb'] = data_csv_raw_ics.loc[3*item, 'segmentation']
    data_csv.loc[item, 'segmentation_sb'] = data_csv_raw_ics.loc[3*item + 1, 'segmentation']
    data_csv.loc[item, 'segmentation_s'] = data_csv_raw_ics.loc[3*item + 2, 'segmentation']
data_csv

,id,path,segmentation_s,segmentation_lb,segmentation_sb,case_id,day_id,slice_h,slice_w,px_spacing_h,px_spacing_w,slice_id
0,case123_day20_slice_0001,./train\case123\case123_day20\slice_0001_266_2...,NaN,NaN,NaN,123,20,266,266,1.50,1.50,0001
1,case123_day20_slice_0002,./train\case123\case123_day20\slice_0002_266_2...,NaN,NaN,NaN,123,20,266,266,1.50,1.50,0002
2,case123_day20_slice_0003,./train\case123\case123_day20\slice_0003_266_2...,NaN,NaN,NaN,123,20,266,266,1.50,1.50,0003
3,case123_day20_slice_0004,./train\case123\case123_day20\slice_0004_266_2...,NaN,NaN,NaN,123,20,266,266,1.50,1.50,0004
4,case123_day20_slice_0005,./train\case123\case123_day20\slice_0005_266_2...,NaN,NaN,NaN,123,20,266,266,1.50,1.50,0005
...,...,...,...,...,...,...,...,...,...,...,...,...
38491,case30_day0_slice_0140,./train\case30\case30_day0\slice_0140_266_266_...,NaN,NaN,NaN,30,0,266,266,1.50,1.50,0140
38492,case30_day0_slice_0141,./train\case30\case30_day0\slice_0141_266_266_...,NaN,NaN,NaN,30,0,266,266,1.50,1.50,0141
38493,case30_day0_slice_0142,./train\case30\case30_day0\slice_0142_266_266_...,NaN,NaN,NaN,30,0,266,266,1.50,1.50,0142
38494,case30_day0_slice_0143,./train\case30\case30_day0\slice_0143_266_266_...,NaN,NaN,NaN,30,0,266,266,1.50,1.50,0143


In [11]:
# 验证是否正确
data_csv.loc[data_csv['id'] == 'case123_day20_slice_0075', ['segmentation_s', 'segmentation_sb', 'segmentation_lb']]

,segmentation_s,segmentation_sb,segmentation_lb
74,12388 11 12652 14 12916 18 13180 22 13445 24 1...,NaN,23058 4 23323 6 23588 8 23854 9 24119 10 24385...


In [12]:
data_csv.to_csv(os.path.join('./', 'data_csv.csv'), index=False)

### data_csv分析

In [1]:
# 导入库
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.offline as of

In [2]:
data_csv = pd.read_csv('./data_csv.csv')
data_csv

,id,path,segmentation_s,segmentation_lb,segmentation_sb,case_id,day_id,slice_h,slice_w,px_spacing_h,px_spacing_w,slice_id
0,case123_day20_slice_0001,./train\case123\case123_day20\slice_0001_266_2...,NaN,NaN,NaN,123,20,266,266,1.5,1.5,1
1,case123_day20_slice_0002,./train\case123\case123_day20\slice_0002_266_2...,NaN,NaN,NaN,123,20,266,266,1.5,1.5,2
2,case123_day20_slice_0003,./train\case123\case123_day20\slice_0003_266_2...,NaN,NaN,NaN,123,20,266,266,1.5,1.5,3
3,case123_day20_slice_0004,./train\case123\case123_day20\slice_0004_266_2...,NaN,NaN,NaN,123,20,266,266,1.5,1.5,4
4,case123_day20_slice_0005,./train\case123\case123_day20\slice_0005_266_2...,NaN,NaN,NaN,123,20,266,266,1.5,1.5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
38491,case30_day0_slice_0140,./train\case30\case30_day0\slice_0140_266_266_...,NaN,NaN,NaN,30,0,266,266,1.5,1.5,140
38492,case30_day0_slice_0141,./train\case30\case30_day0\slice_0141_266_266_...,NaN,NaN,NaN,30,0,266,266,1.5,1.5,141
38493,case30_day0_slice_0142,./train\case30\case30_day0\slice_0142_266_266_...,NaN,NaN,NaN,30,0,266,266,1.5,1.5,142
38494,case30_day0_slice_0143,./train\case30\case30_day0\slice_0143_266_266_...,NaN,NaN,NaN,30,0,266,266,1.5,1.5,143


In [32]:
# case数量图
case = data_csv['case_id'].value_counts(ascending=True)
trace1 = go.Scatter(
    x=list(case.index),
    y=list(case.values),
    mode = 'markers',
    name = 'case_num'
)

In [33]:
# slice_id数量图
slice_id = data_csv['slice_id'].value_counts()
trace2 = go.Scatter(
    x=list(slice_id.index),
    y=list(slice_id.values),
    mode = 'markers',
    name = 'slice_num'
)
fig = go.Figure(data=[trace1, trace2])
fig.show()

In [41]:
# day_id数量图
day_id = data_csv['day_id'].value_counts()
trace3 = go.Bar(
    x=list(day_id.index),
    y=list(day_id.values),
)
layout = go.Layout(
    barmode='stack'
)
fig = go.Figure(data=trace3, layout=layout)
fig.show()